# Saving model

Saving and loading PyTorch models is crucial because any model you build needs to be transferred and deployed in some way. Check the [official tutorial](https://pytorch.org/tutorials/beginner/saving_loading_models.html). Here, we'll experiment with the options from the tutorial.

In [30]:
import torch
from torch import nn

import itertools
from pathlib import Path

## State dict

The classical method to save and load a model's state dictionary follows these steps:

- Retrieve the model's state dictionary with `nn.Module.state_dict()`.
- Save the state dictionary with `torch.save`.
- Load the state dictionary with `torch.load`.
- Load the weights into the model using `nn.Module.load_state_dict()`.

---

In the following cell, we created a simple model and initialized it with a constant value.

In [ ]:
model = nn.Sequential(
    nn.Linear(3, 3),
    nn.Linear(3, 3)
)
for p in model.parameters():
    nn.init.constant_(p, 3)

model.state_dict()

OrderedDict([('0.weight',
              tensor([[3., 3., 3.],
                      [3., 3., 3.],
                      [3., 3., 3.]])),
             ('0.bias', tensor([3., 3., 3.])),
             ('1.weight',
              tensor([[3., 3., 3.],
                      [3., 3., 3.],
                      [3., 3., 3.]])),
             ('1.bias', tensor([3., 3., 3.]))])

Now, using `torch.save`, we save the state dictionary and discard the original model.

In [ ]:
torch.save(obj=model.state_dict(), f=Path("/tmp")/"my_model")
del model

Now, with `torch.load`, we load the state dictionary—since all values were constant during saving, they remain as 3.

In [ ]:
state_dict = torch.load(Path("/tmp")/"my_model", weights_only=False)
state_dict

OrderedDict([('0.weight',
              tensor([[3., 3., 3.],
                      [3., 3., 3.],
                      [3., 3., 3.]])),
             ('0.bias', tensor([3., 3., 3.])),
             ('1.weight',
              tensor([[3., 3., 3.],
                      [3., 3., 3.],
                      [3., 3., 3.]])),
             ('1.bias', tensor([3., 3., 3.]))])

By recreating the exact same model and loading the previously saved state dictionary into it, you can fully recreate the model.

In [ ]:
model = nn.Sequential(
    nn.Linear(3, 3),
    nn.Linear(3, 3)
)

model.load_state_dict(state_dict)

<All keys matched successfully>

Successfully executed `nn.load_state_dict` returns a special string.

## Save entire model

By passing the entire model to the `torch.save` function, you'll save a serialized Torch model. Then, with just one line of code, you can restore the model using the `torch.load` function.

---

The following cell demonstrates creating a model, initializing its weights with a constant, and saving this model to disk by passing it as `obj` to the `torch.save` function.

In [ ]:
model = nn.Sequential(
    nn.Linear(3, 3),
    nn.Linear(3, 3)
)
for p in model.parameters():
    nn.init.constant_(p, 3)

torch.save(model, Path("/tmp")/"model.pht")

With `torch.load`, the model can be restored. The following cell shows that the loaded model has weights identical to those initialized before saving.

In [ ]:
torch.load(Path("/tmp")/"model.pht", weights_only=False).state_dict()

OrderedDict([('0.weight',
              tensor([[3., 3., 3.],
                      [3., 3., 3.],
                      [3., 3., 3.]])),
             ('0.bias', tensor([3., 3., 3.])),
             ('1.weight',
              tensor([[3., 3., 3.],
                      [3., 3., 3.],
                      [3., 3., 3.]])),
             ('1.bias', tensor([3., 3., 3.]))])

## Saving group

A typical case where this approach requires two `torch` objects, such as a generator and discriminator in a GAN, is straightforward. It's not a problem to save a batch of models grouped in any Python collection. 

---

As an example, consider two linear layers that we want to save as a single file.

In [31]:
val1 = nn.Linear(3, 3)
val2 = nn.Linear(3, 3)

for p in itertools.chain(val1.parameters(), val2.parameters()):
    nn.init.constant_(p, 3)

We'll save models as dictionary.

In [23]:
torch.save(
    obj={"val1": val1, "val2": val2},
    f=Path("/tmp")/"model.pth"
)

So there are no problems to load that dictionary:

In [19]:
ans = torch.load(Path("/tmp")/"model.pth", weights_only=False)
ans

{'val1': Linear(in_features=3, out_features=3, bias=True),
 'val2': Linear(in_features=3, out_features=3, bias=True)}

To be sure, let's check that the extracted model has weights just as we initialized them before saving.

In [21]:
ans["val1"].state_dict()

OrderedDict([('weight',
              tensor([[3., 3., 3.],
                      [3., 3., 3.],
                      [3., 3., 3.]])),
             ('bias', tensor([3., 3., 3.]))])

The same trick will work with tuples as well.

In [28]:
torch.save(obj=(val1, val2), f=Path("/tmp")/"model.pth")
torch.load(Path("/tmp")/"model.pth", weights_only=False)

(Linear(in_features=3, out_features=3, bias=True),
 Linear(in_features=3, out_features=3, bias=True))